**Code to read a csv file that contains the area of the fileds in acres and converts it to pixels**

authors: Pol Arranz-Gibert

Import the necessary libraries:
- Pandas: Read and write csv files
- Numpy: Numerical Computations

In [1]:
import pandas as pd
import numpy as np

Read the meta-data file **train-unique.csv** for the training images and **test.csv** for the test images.

In [2]:
train = pd.read_csv('../maizefinder/d/train-unique.csv')
test = pd.read_csv('../maizefinder/d/test.csv')

Quick analysis of the train meta-data file that includes shape, initial and end part of the file, and distribution by field size.

In [3]:
print("Train meta-data shape:", train.shape, "\n")
print(train.head(5), "\n")
print(train.tail(5), "\n")
print(train.PlotSize_acres.value_counts())

Train meta-data shape: (366, 7) 

            ID  Year  PlotSize_acres    Yield  Quality         x         y
0  id_ade6610c  2018            0.25      NaN        2 -0.100546  0.131002
1  id_c987f14e  2018            2.00      NaN        1 -0.985732 -0.032197
2  id_92e2cdbf  2016            1.00  0.37125        1 -1.335971 -1.069998
3  id_a1ce519e  2017            1.50  0.26875        3  0.000000  0.000000
4  id_fe53ff60  2018            0.25      NaN        1  0.173911  0.273041 

              ID  Year  PlotSize_acres     Yield  Quality         x         y
361  id_387fcf14  2015            1.00  0.265000        2  0.169148  0.101470
362  id_34b4d068  2015            0.40  0.193125        2 -0.087902 -0.027066
363  id_7940ea69  2015            0.50  0.187500        2 -0.289853  0.108825
364  id_1a45efa2  2015            0.75  0.567500        1 -0.629659  0.127603
365  id_3df95f62  2015            1.00  0.368750        2 -0.182798 -0.191495 

0.50000    123
0.25000     81
1.00000     57

Quick analysis of the test meta-data file that includes shape, initial and end part of the file, and distribution by field size.

In [4]:
print("Train meta-data shape:", test.shape, "\n")
print(test.head(5), "\n")
print(test.tail(5), "\n")
print(test.PlotSize_acres.value_counts())

Train meta-data shape: (1613, 4) 

            ID  Year  PlotSize_acres  Yield
0  id_e7032b10  2016            0.50    0.0
1  id_ae7cb51e  2018            1.00    NaN
2  id_e59f7730  2018            1.00    NaN
3  id_b9011c86  2018            1.50    NaN
4  id_caaeb9f8  2018            0.25    NaN 

               ID  Year  PlotSize_acres     Yield
1608  id_36ec7a7b  2018            0.25       NaN
1609  id_70f3d8f7  2018            1.00       NaN
1610  id_2b7f2b96  2017            0.90  0.591875
1611  id_498f3db0  2018            1.75       NaN
1612  id_fe2a6c59  2018            1.50       NaN 

0.500000     543
0.250000     433
1.000000     180
0.125000      99
0.750000      97
1.500000      39
2.000000      34
1.250000      21
0.200000      17
0.300000      12
3.000000      11
0.375000       9
0.700000       9
0.062500       8
5.000000       7
0.400000       7
0.625000       7
0.100000       6
2.500000       5
1.125000       4
0.150000       4
0.800000       4
0.900000       3
4.0000

We define a function to convert acres to pixels for both types of images (from Planet and Sentinel-2) using any of the meta-data files (**train-unique.csv** or **text.csv**).

In [5]:
def acres_to_px(file):

    file2 = file.set_index("ID", drop = False)

    num_rows = file2.shape[0]

    # 1 acre = 4046.86 m2
    # Planet Lab Satellite in ~4.7 m resolution
    # Sentinel-2 Satellite in ~10 m resolution
    # to be used as X acres * 4046.86 m2/acre (1/RESOLUTION) pixel/m2
    acres_to_px_factor_Planet = 4046.86//(4.7) 
    acres_to_px_factor_Sentinel = 4046.86//(10) 

    i = 0
    PlotSize_px_Planet = np.ones((num_rows, 1))
    PlotSize_px_Sentinel = np.ones((num_rows, 1))

    for ID in file2["ID"]:
        PlotSize_px_Planet [i] = file2.loc[ID, "PlotSize_acres"] * acres_to_px_factor_Planet
        PlotSize_px_Sentinel [i] = file2.loc[ID, "PlotSize_acres"] * acres_to_px_factor_Sentinel
        i = i+1

    file2["PlotSize_px_Planet"] = PlotSize_px_Planet
    file2["PlotSize_px_Sentinel"] = PlotSize_px_Sentinel

    return file2

Visualize the new meta-data file for the training set. It includes two columns containing the number of pixels for Planet and Sentinel datasets.

In [6]:
acres_to_px(train)

,ID,Year,PlotSize_acres,Yield,Quality,x,y,PlotSize_px_Planet,PlotSize_px_Sentinel
ID,,,,,,,,,
id_ade6610c,id_ade6610c,2018,0.25,NaN,2,-0.100546,0.131002,215.25,101.0
id_c987f14e,id_c987f14e,2018,2.00,NaN,1,-0.985732,-0.032197,1722.00,808.0
id_92e2cdbf,id_92e2cdbf,2016,1.00,0.371250,1,-1.335971,-1.069998,861.00,404.0
id_a1ce519e,id_a1ce519e,2017,1.50,0.268750,3,0.000000,0.000000,1291.50,606.0
id_fe53ff60,id_fe53ff60,2018,0.25,NaN,1,0.173911,0.273041,215.25,101.0
...,...,...,...,...,...,...,...,...,...
id_387fcf14,id_387fcf14,2015,1.00,0.265000,2,0.169148,0.101470,861.00,404.0
id_34b4d068,id_34b4d068,2015,0.40,0.193125,2,-0.087902,-0.027066,344.40,161.6
id_7940ea69,id_7940ea69,2015,0.50,0.187500,2,-0.289853,0.108825,430.50,202.0


Visualize the new meta-data file for the test set. It includes two columns containing the number of pixels for Planet and Sentinel datasets.

In [7]:
acres_to_px(test)

,ID,Year,PlotSize_acres,Yield,PlotSize_px_Planet,PlotSize_px_Sentinel
ID,,,,,,
id_e7032b10,id_e7032b10,2016,0.50,0.000000,430.50,202.0
id_ae7cb51e,id_ae7cb51e,2018,1.00,NaN,861.00,404.0
id_e59f7730,id_e59f7730,2018,1.00,NaN,861.00,404.0
id_b9011c86,id_b9011c86,2018,1.50,NaN,1291.50,606.0
id_caaeb9f8,id_caaeb9f8,2018,0.25,NaN,215.25,101.0
...,...,...,...,...,...,...
id_36ec7a7b,id_36ec7a7b,2018,0.25,NaN,215.25,101.0
id_70f3d8f7,id_70f3d8f7,2018,1.00,NaN,861.00,404.0
id_2b7f2b96,id_2b7f2b96,2017,0.90,0.591875,774.90,363.6
